In [2]:
import cv2
import numpy as np
from imutils.object_detection import non_max_suppression

east_model_path = "frozen_east_text_detection.pb"
net = cv2.dnn.readNet(east_model_path)

def detect_text(image):
    orig = image.copy()
    (H, W) = image.shape[:2]
    
    new_w, new_h = (320, 320)
    rW, rH = W / float(new_w), H / float(new_h)
    image = cv2.resize(image, (new_w, new_h))

    layer_names = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
    blob = cv2.dnn.blobFromImage(image, 1.0, (320, 320), (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layer_names)

    return orig, scores, geometry, rW, rH

def decode_predictions(scores, geometry, min_confidence=0.5):
    num_rows, num_cols = scores.shape[2:4]
    rects, confidences = [], []

    for y in range(num_rows):
        for x in range(num_cols):
            score = scores[0, 0, y, x]
            if score < min_confidence:
                continue

            offset_x, offset_y = x * 4.0, y * 4.0
            angle = geometry[0, 4, y, x]
            cos, sin = np.cos(angle), np.sin(angle)
            h, w = geometry[0, 0, y, x], geometry[0, 1, y, x]

            end_x = int(offset_x + (cos * w) + (sin * h))
            end_y = int(offset_y - (sin * w) + (cos * h))
            start_x, start_y = int(end_x - w), int(end_y - h)

            rects.append((start_x, start_y, end_x, end_y))
            confidences.append(float(score))

    return rects, confidences

def apply_nms(rects, confidences, rW, rH):
    boxes = np.array(rects)
    boxes = non_max_suppression(boxes, probs=confidences)

    scaled_boxes = [(int(start_x * rW), int(start_y * rH), int(end_x * rW), int(end_y * rH)) for (start_x, start_y, end_x, end_y) in boxes]
    return scaled_boxes
